In [32]:
# Imports
import numpy as np
import datetime,os
import pandas as pd
from fuzzywuzzy import fuzz
import pyodbc

# Assembles an appropriate SQL write query
def assemble_sql_write_query(table_name,row):
    sql_write_query = 'INSERT INTO '+table_name+'('
    for colname in row.index:
        sql_write_query += '['
        sql_write_query += colname
        sql_write_query += '],'
    sql_write_query = sql_write_query[:-1]
    sql_write_query += ') VALUES('
    for colname in row.index:
        sql_write_query += '\''
        sql_write_query += str(row[colname])
        sql_write_query += '\''
        sql_write_query += ','
    sql_write_query = sql_write_query[:-1]
    sql_write_query += ')'
    return sql_write_query

# Clears DB of table, and replaces it with new table stored in pandas format
def pandas_to_access_clean_write(conn,df,table_name,verbose=False):
    cursor = conn.cursor()
    
    sql_delete_query = 'DELETE * FROM '+table_name
    cursor.execute(sql_delete_query)
    conn.commit()
    
    for index,row in df.iterrows():
        sql_write_query = assemble_sql_write_query(table_name,row)
        if verbose: print(sql_write_query+'\n\n')
        cursor.execute(sql_write_query)
        conn.commit()
    cursor.close()
    
    return

# Clear access DB of table
def access_clear_table(conn,table_name):

    cursor = conn.cursor()

    sql_delete_query = 'DELETE * FROM '+table_name
    cursor.execute(sql_delete_query)
    conn.commit()

    cursor.close()
    
    return

# Find a row in STR file
def find_row_in_STR(df,of,df_idx,THRESHOLD=0.01):
    assert df_idx in df.index,'Index not in data frame.'
    assert set(df.columns).issubset(set(of.columns)),'Some required columns were not found in table file.'
    of = of[df.columns.tolist()]
    assert all([df[c_col].dtype==of[c_col].dtype for c_col in of.columns]),'Found non-matching column types between comparison data frames.'
    rl = []
    for c_idx in of.index:
        tl = []
        exceeds_threshold = False
        for c_col in of.columns:
            if df[c_col].dtype=='datetime64[ns]':
                c_div = np.abs((df.iloc[df_idx][c_col]-of.iloc[c_idx][c_col]).days)
            elif df[c_col].dtype=='object':
                c_div = float((100-fuzz.ratio(str(df.iloc[df_idx][c_col]),str(of.iloc[c_idx][c_col])))/100)
            elif df[c_col].dtype=='float64':
                c_div = np.abs(df.iloc[df_idx][c_col]-of.iloc[c_idx][c_col])
            else:
                raise Exception('Column name does not match expected format.')
            if c_div>THRESHOLD: 
                exceeds_threshold = True
                break
            tl.append(c_div)
        if not exceeds_threshold: rl.append([c_idx,tl])
    return rl

# Find all matches between STR and pandas frame
def DF_STR_find_matches(df,STR):
    matches = []
    for df_idx in df.index:
        res = find_row_in_STR(df,STR,df_idx)
        if len(res)==0:
            matches.append([df_idx,int(-1)])
        elif len(res)==1:
            matches.append([df_idx,int(res[0][0])])
        else: 
            print(res)
            matches.append([df_idx,int(-2)])
    matches = pd.DataFrame(matches).rename(columns={0:'DF index',1:'STR index'}).set_index('DF index')
    return matches

# Read in new data
def read_newdata(dirpath):
    # Read new data
    bf = pd.DataFrame()
    for file in os.listdir(dirpath):
        tf = pd.read_excel(dirpath+file)
        bf = pd.concat([bf,tf],sort=False)
    # Transform new data
    bf['Buchungstag'] = pd.to_datetime(bf['Buchungstag'],format='%d.%m.%y')
    bf['Valutadatum'] = pd.to_datetime(bf['Valutadatum'],format='%d.%m.%y')
    # Transform base data
    df = bf[['Buchungstag','Valutadatum','Buchungstext','Verwendungszweck','Beguenstigter/Zahlungspflichtiger','Kontonummer/IBAN','Betrag']].rename(columns={
        'Valutadatum':'Date_ordered', 
        'Buchungstag':'Date_booked', 
        'Buchungstext':'Text_transaction', 
        'Verwendungszweck':'Use',
        'Beguenstigter/Zahlungspflichtiger':'Contact', 
        'Kontonummer/IBAN':'AccNum', 
        'Betrag':'Value_transaction'
    })
    df.index = df.index.rename('STR_id')
    # Transform added information
    af = bf[['Auftragskonto','Glaeubiger ID','Mandatsreferenz','Kundenreferenz (End-to-End)','Sammlerreferenz','Lastschrift Ursprungsbetrag','Auslagenersatz Ruecklastschrift','BIC (SWIFT-Code)','Waehrung','Info']].rename(columns={
        'Auftragskonto':'OrderAccount', 
        'Glaeubiger ID':'LenderID', 
        'Mandatsreferenz':'MandateReference',
        'Kundenreferenz (End-to-End)':'CustomerReference', 
        'Sammlerreferenz':'CollectorReference',
        'Lastschrift Ursprungsbetrag':'Amount0', 
        'Auslagenersatz Ruecklastschrift':'Amount1',
        'BIC (SWIFT-Code)':'BIC', 
        'Waehrung':'Currency', 
        'Info':'Information'
    })
    af.index = af.index.rename('STR_id')
    return bf,df,af

# Read in STR table
def read_STR(dbpath):
    # Connect to Access DB
    conn = pyodbc.connect(r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ='+dbpath+';')
    # Read Single Transactions table
    STR = pd.read_sql_query(
    '''
    SELECT * FROM STR
    '''
    ,conn).set_index('STR_id').reset_index(drop=True)
    # Close connection
    conn.close()
    # Transform STR
    STR['Date_booked'] = pd.to_datetime(STR['Date_booked'])
    STR['Date_ordered'] = pd.to_datetime(STR['Date_ordered'])
    STR['Value_transaction'] = STR['Value_transaction'].astype('float')
    return STR

# Read in TRI table
def read_TRI(dbpath):
    # Connect to Access DB
    conn = pyodbc.connect(r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ='+dbpath+';')
    # Read Transaction Information table
    TRI = pd.read_sql_query(
    '''
    SELECT * FROM TRINFORMATION
    '''
    ,conn).set_index('STR_id').reset_index(drop=True)
    # Close connection
    conn.close()
    return TRI

# Main function
def __main__():
    # Path definitions
    newdatapath = '..\\..\\..\\TF_data\\ADMIN\\01\\'
    dbpath = '..\\..\\..\\..\\banking.accdb'
    # Read new input
    bf,df,af = read_newdata(newdatapath)
    # Read existing databases
    STR = read_STR(dbpath)
    TRI = read_TRI(dbpath)
    # Detect existing entries
    matches = DF_STR_find_matches(df,STR)
    # Select new data
    new_data = df.loc[matches[matches['STR index']<0].index]
    new_data['SCA_id'] = 'None'
    # Select new added information
    new_added_information = af.loc[matches[matches['STR index']<0].index]
    # Make new files
    STR_new = pd.concat([STR,new_data],sort=False).reset_index(drop=True)
    TRI_new = pd.concat([TRI,new_added_information],sort=False).reset_index(drop=True)
    # Connect to Access DB
    conn = pyodbc.connect(r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ='+dbpath+';')
    # Modify DB STR table
    pandas_to_access_clean_write(conn,STR_new,'STR')
    # Modify DB TRI table
    pandas_to_access_clean_write(conn,TRI_new,'TRINFORMATION')
    # Close connection
    conn.close()
    return

In [31]:
__main__()

[[130, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.0]], [345, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.0]]]
[[131, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.0]], [346, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.0]]]
[[132, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.0]], [347, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.0]]]
[[133, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.0]], [348, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.0]]]
[[134, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.0]], [349, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.0]]]
[[135, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.0]], [350, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.0]]]
[[136, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.0]], [351, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.0]]]
[[137, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.0]], [352, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.0]]]
[[138, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.0]], [353, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.0]]]
[[139, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.0]], [354, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.0]]]
[[140, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.0]], [355, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.0]]]
[[141, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.0]], [356, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.0]]]
[[142, [0, 0, 0.0, 0.0, 0.0,

INSERT INTO STR([Date_booked],[Date_ordered],[Text_transaction],[Use],[Contact],[AccNum],[Value_transaction],[SCA_id]) VALUES('2019-08-23 00:00:00','2019-08-23 00:00:00','KARTENZAHLUNG','2019-08-22T17:17 Debitk.5 2021-12 ','YORMA S AG//PLATTLING/DE','DE89741200710006466257','-5.0','None')


INSERT INTO STR([Date_booked],[Date_ordered],[Text_transaction],[Use],[Contact],[AccNum],[Value_transaction],[SCA_id]) VALUES('2019-08-23 00:00:00','2019-08-23 00:00:00','KARTENZAHLUNG','2019-08-22T17:54 Debitk.5 2021-12 ','ALDI GMBH + CO. KG//ERLANGEN/DE','DE29300500000001132513','-62.0','None')


INSERT INTO STR([Date_booked],[Date_ordered],[Text_transaction],[Use],[Contact],[AccNum],[Value_transaction],[SCA_id]) VALUES('2019-08-22 00:00:00','2019-08-22 00:00:00','EIGENE KREDITKARTENABRECHN.','VISA-ABR. 490762XXXXXX5575 ','KREDITKARTENABRECHNUNG','9902292060','-345.0','None')


INSERT INTO STR([Date_booked],[Date_ordered],[Text_transaction],[Use],[Contact],[AccNum],[Value_transaction],[SCA_id]) VA

INSERT INTO STR([Date_booked],[Date_ordered],[Text_transaction],[Use],[Contact],[AccNum],[Value_transaction],[SCA_id]) VALUES('2019-07-10 00:00:00','2019-07-10 00:00:00','FOLGELASTSCHRIFT','D01-9646851-5179845 Prime Video 5PW3VA2DBWO9AY7C ','AMAZON INSTANT VIDEO GERMANY GMBH','DE07300308800013011001','-4.0','None')


INSERT INTO STR([Date_booked],[Date_ordered],[Text_transaction],[Use],[Contact],[AccNum],[Value_transaction],[SCA_id]) VALUES('2019-07-09 00:00:00','2019-07-09 00:00:00','FOLGELASTSCHRIFT','. SPOTIFY, Ihr Einkauf bei SPOTIFY ','PayPal (Europe) S.a.r.l. et Cie., S.C.A.','DE88500700100175526303','-10.0','None')


INSERT INTO STR([Date_booked],[Date_ordered],[Text_transaction],[Use],[Contact],[AccNum],[Value_transaction],[SCA_id]) VALUES('2019-07-08 00:00:00','2019-07-08 00:00:00','FOLGELASTSCHRIFT','Kieser Training-Beitrag (ID238910) VOM 03.07.2019 ','JENS MOELLER GMBH','DE85763500000000001669','-69.0','None')


INSERT INTO STR([Date_booked],[Date_ordered],[Text_transaction]

INSERT INTO STR([Date_booked],[Date_ordered],[Text_transaction],[Use],[Contact],[AccNum],[Value_transaction],[SCA_id]) VALUES('2019-05-31 00:00:00','2019-05-31 00:00:00','FOLGELASTSCHRIFT','VST 458207, 67,00 EUR A19-347116 ','ESTW-ERLANGERSTADTWERKEAG','DE06763200720004536703','-67.0','None')


INSERT INTO STR([Date_booked],[Date_ordered],[Text_transaction],[Use],[Contact],[AccNum],[Value_transaction],[SCA_id]) VALUES('2019-05-29 00:00:00','2019-05-29 00:00:00','DAUERAUFTRAG','Miete fuer Haesschen, du bist toll :) ','Elaine Fernandez','DE46763500000044116613','-235.0','None')


INSERT INTO STR([Date_booked],[Date_ordered],[Text_transaction],[Use],[Contact],[AccNum],[Value_transaction],[SCA_id]) VALUES('2019-06-18 00:00:00','2019-06-18 00:00:00','FOLGELASTSCHRIFT','D01-8033778-8722238 Prime Video 6MCG5V6Y45G5J5BY ','AMAZON INSTANT VIDEO GERMANY GMBH','DE07300308800013011001','-399.0','None')


INSERT INTO STR([Date_booked],[Date_ordered],[Text_transaction],[Use],[Contact],[AccNum],[Valu

INSERT INTO STR([Date_booked],[Date_ordered],[Text_transaction],[Use],[Contact],[AccNum],[Value_transaction],[SCA_id]) VALUES('2019-05-15 00:00:00','2019-05-15 00:00:00','FOLGELASTSCHRIFT','K522464184 Beitrag 01.04.2019 30.04.2019 96.19 ','Siemens BKK','DE81700800000354038800','-9619.0','None')


INSERT INTO STR([Date_booked],[Date_ordered],[Text_transaction],[Use],[Contact],[AccNum],[Value_transaction],[SCA_id]) VALUES('2019-05-15 00:00:00','2019-05-15 00:00:00','FOLGELASTSCHRIFT','D-Nr 1010002015, /INV/B000000051608 vom 08.05.2019, Rechnung, ZahlB 1301490953 ','M-net Telekommunikations GmbH','DE74700500000000028273','-2990.0','None')


INSERT INTO STR([Date_booked],[Date_ordered],[Text_transaction],[Use],[Contact],[AccNum],[Value_transaction],[SCA_id]) VALUES('2019-05-13 00:00:00','2019-05-11 00:00:00','BARGELDAUSZAHLUNG','2019-05-11T11:48 Debitk.5 2021-12 ','NBG. STR.//SPARKASSE ERLANGEN/DE','DE63763500009000481424','-3000.0','None')


INSERT INTO STR([Date_booked],[Date_ordered],[T

...

INSERT INTO STR([Date_booked],[Date_ordered],[Text_transaction],[Use],[Contact],[AccNum],[Value_transaction],[SCA_id]) VALUES('2019-06-11 00:00:00','2019-06-10 00:00:00','KARTENZAHLUNG','2019-06-07T12:28 Debitk.5 2021-12 ','ALDI GMBH + CO. KG//ERLANGEN/DE','DE29300500000001132513','-2.68','None')


INSERT INTO STR([Date_booked],[Date_ordered],[Text_transaction],[Use],[Contact],[AccNum],[Value_transaction],[SCA_id]) VALUES('2019-07-25 00:00:00','2019-07-25 00:00:00','FOLGELASTSCHRIFT','D01-8332720-6610260 Prime Video 1PJUZ9P3PYJZEDKI ','AMAZON INSTANT VIDEO GERMANY GMBH','DE07300308800013011001','-399.0','None')


INSERT INTO STR([Date_booked],[Date_ordered],[Text_transaction],[Use],[Contact],[AccNum],[Value_transaction],[SCA_id]) VALUES('2019-06-06 00:00:00','2019-06-06 00:00:00','FOLGELASTSCHRIFT','D01-5445295-4040669 Prime Video 5VLB84D81P4SHPTL ','AMAZON INSTANT VIDEO GERMANY GMBH','DE07300308800013011001','-3.99','None')


INSERT INTO STR([Date_booked],[Date_ordered],[Text_transact


INSERT INTO STR([Date_booked],[Date_ordered],[Text_transaction],[Use],[Contact],[AccNum],[Value_transaction],[SCA_id]) VALUES('2019-06-05 00:00:00','2019-06-05 00:00:00','FOLGELASTSCHRIFT','D01-0715639-1736631 Prime Video 6RER9EEMXZ5T0F2Q ','AMAZON INSTANT VIDEO GERMANY GMBH','DE07300308800013011001','-399.0','None')


INSERT INTO STR([Date_booked],[Date_ordered],[Text_transaction],[Use],[Contact],[AccNum],[Value_transaction],[SCA_id]) VALUES('2019-06-05 00:00:00','2019-06-05 00:00:00','FOLGELASTSCHRIFT','Kieser Training-Beitrag (ID238909) VOM 03.06.2019 ','JENS MOELLER GMBH','DE85763500000000001669','-6900.0','None')


INSERT INTO STR([Date_booked],[Date_ordered],[Text_transaction],[Use],[Contact],[AccNum],[Value_transaction],[SCA_id]) VALUES('2019-06-04 00:00:00','2019-06-04 00:00:00','FOLGELASTSCHRIFT','D01-7024908-5090217 Prime Video 46MBH24MG8R9STAW ','AMAZON INSTANT VIDEO GERMANY GMBH','DE07300308800013011001','-399.0','None')


INSERT INTO STR([Date_booked],[Date_ordered],[Text


INSERT INTO STR([Date_booked],[Date_ordered],[Text_transaction],[Use],[Contact],[AccNum],[Value_transaction],[SCA_id]) VALUES('2019-05-02 00:00:00','2019-05-02 00:00:00','GUTSCHR. UEBERW. DAUERAUFTR','TASCHENGELD ','Hermann Kuespert','DE80773501100020085395','20000.0','None')


INSERT INTO STR([Date_booked],[Date_ordered],[Text_transaction],[Use],[Contact],[AccNum],[Value_transaction],[SCA_id]) VALUES('2019-04-30 00:00:00','2019-05-01 00:00:00','ENTGELTABSCHLUSS','Entgeltabrechnung siehe Anlage ','nan','0000000000','-450.0','None')


INSERT INTO STR([Date_booked],[Date_ordered],[Text_transaction],[Use],[Contact],[AccNum],[Value_transaction],[SCA_id]) VALUES('2019-04-30 00:00:00','2019-04-30 00:00:00','FOLGELASTSCHRIFT','VST 458207, 67,00 EUR A19-273596 ','ESTW-ERLANGERSTADTWERKEAG','DE06763200720004536703','-6700.0','None')


INSERT INTO STR([Date_booked],[Date_ordered],[Text_transaction],[Use],[Contact],[AccNum],[Value_transaction],[SCA_id]) VALUES('2019-04-30 00:00:00','2019-04-30 0

INSERT INTO STR([Date_booked],[Date_ordered],[Text_transaction],[Use],[Contact],[AccNum],[Value_transaction],[SCA_id]) VALUES('2019-03-18 00:00:00','2019-03-18 00:00:00','FOLGELASTSCHRIFT','Reise-VSNR 783299846 14.03.2019 ','HANSEMERKUR VERS.','DE41200300000000223842','-990.0','None')


INSERT INTO STR([Date_booked],[Date_ordered],[Text_transaction],[Use],[Contact],[AccNum],[Value_transaction],[SCA_id]) VALUES('2019-03-15 00:00:00','2019-03-15 00:00:00','FOLGELASTSCHRIFT','TK-BuchNr 05204061749 Monat 02/19 K522464184 Beitraege ','Techniker Krankenkasse','DE18200100200030936201','-9229.0','None')


INSERT INTO STR([Date_booked],[Date_ordered],[Text_transaction],[Use],[Contact],[AccNum],[Value_transaction],[SCA_id]) VALUES('2019-03-14 00:00:00','2019-03-14 00:00:00','FOLGELASTSCHRIFT','D-Nr 1010002015, /INV/B000000042174 vom 07.03.2019, Rechnung, ZahlB 1300744806 ','M-net Telekommunikations GmbH','DE74700500000000028273','-2990.0','None')


INSERT INTO STR([Date_booked],[Date_ordered],[T

INSERT INTO STR([Date_booked],[Date_ordered],[Text_transaction],[Use],[Contact],[AccNum],[Value_transaction],[SCA_id]) VALUES('2019-01-02 00:00:00','2019-01-02 00:00:00','BARGELDAUSZAHLUNG','2019-01-02T12:47 Debitk.5 2021-12 ','ROETHELH.//SPARKASSE ERLANGEN/DE','DE63763500009000481424','-5000.0','None')


INSERT INTO STR([Date_booked],[Date_ordered],[Text_transaction],[Use],[Contact],[AccNum],[Value_transaction],[SCA_id]) VALUES('2019-01-02 00:00:00','2019-01-02 00:00:00','GUTSCHR. UEBERW. DAUERAUFTR','TASCHENGELD ','Hermann Kuespert','DE80773501100020085395','20000.0','None')


INSERT INTO STR([Date_booked],[Date_ordered],[Text_transaction],[Use],[Contact],[AccNum],[Value_transaction],[SCA_id]) VALUES('2019-01-02 00:00:00','2018-12-31 00:00:00','FOLGELASTSCHRIFT','301218 14459120 610971995391 SIEMENS CASINO CARD NOVEMBER 2018 SRE R+S Restaurant Services ','Siemens Aktiengesellschaft','DE53700700100203008800','-5466.0','None')


INSERT INTO STR([Date_booked],[Date_ordered],[Text_transa

In [33]:
newdatapath = '..\\..\\..\\TF_data\\ADMIN\\01\\'
dbpath = '..\\..\\..\\..\\banking.accdb'
# Read new input
bf,df,af = read_newdata(newdatapath)
# Read existing databases
STR = read_STR(dbpath)
TRI = read_TRI(dbpath)
# Detect existing entries
matches = DF_STR_find_matches(df,STR)
# Select new data
new_data = df.loc[matches[matches['STR index']<0].index]
new_data['SCA_id'] = 'None'
# Select new added information
new_added_information = af.loc[matches[matches['STR index']<0].index]
# Make new files
STR_new = pd.concat([STR,new_data],sort=False).reset_index(drop=True)
TRI_new = pd.concat([TRI,new_added_information],sort=False).reset_index(drop=True)
# Connect to Access DB
conn = pyodbc.connect(r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ='+dbpath+';')
# Modify DB STR table
pandas_to_access_clean_write(conn,STR_new,'STR',verbose=True)
# Modify DB TRI table
pandas_to_access_clean_write(conn,TRI_new,'TRINFORMATION')
# Close connection
conn.close()

[[10, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.009999999999999787]], [538, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.009999999999999787]], [603, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.009999999999999787]]]
[[11, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.009999999999999787]], [540, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.009999999999999787]], [605, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.009999999999999787]]]
[[14, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.009999999999999787]], [514, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.009999999999999787]], [611, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.009999999999999787]]]
[[17, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.009999999999999787]], [520, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.009999999999999787]], [760, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.009999999999999787]]]
[[20, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.009999999999999787]], [526, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.009999999999999787]], [766, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.009999999999999787]]]
[[23, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.009999999999999787]], [530, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.009999999999999787]], [770, [0, 0, 0.0, 0.0, 0.0, 0.0

[[46, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.009999999999999787]], [645, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.009999999999999787]], [804, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.009999999999999787]]]
[[48, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.009999999999999787]], [649, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.009999999999999787]], [808, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.009999999999999787]]]
[[56, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.009999999999999787]], [659, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.009999999999999787]], [818, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.009999999999999787]]]
[[58, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.009999999999999787]], [661, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.009999999999999787]], [820, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.009999999999999787]]]
[[62, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.009999999999999787]], [665, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.009999999999999787]], [824, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.009999999999999787]]]
[[64, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.009999999999999787]], [669, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.009999999999999787]], [828, [0, 0, 0.0, 0.0, 0.0, 0.0

[[650, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.0]], [809, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.0]]]
[[652, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.0]], [811, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.0]]]
[[654, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.0]], [813, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.0]]]
[[656, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.0]], [815, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.0]]]
[[658, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.0]], [817, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.0]]]
[[660, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.0]], [819, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.0]]]
[[662, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.0]], [821, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.0]]]
[[664, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.0]], [823, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.0]]]
[[666, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.0]], [825, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.0]]]
[[668, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.0]], [827, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.0]]]
[[670, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.0]], [829, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.0]]]
[[672, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.0]], [831, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.0]]]
[[674, [0, 0, 0.0, 0.0, 0.0,

[[146, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.0]], [361, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.0]], [980, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.0]]]
[[147, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.0]], [362, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.0]], [981, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.0]]]
[[148, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.0]], [363, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.0]], [982, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.0]]]
[[149, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.0]], [364, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.0]], [983, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.0]]]
[[150, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.0]], [365, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.0]], [984, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.0]]]
[[151, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.0]], [366, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.0]], [985, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.0]]]
[[152, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.0]], [367, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.0]], [986, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.0]]]
[[153, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.0]], [368, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.0]], [987, [0, 0, 0.0, 0.0, 0.0, 0.0, 0.0]]]
[[154, [0, 0, 0.0, 0.0, 0.0, 0.0

INSERT INTO STR([Date_booked],[Date_ordered],[Text_transaction],[Use],[Contact],[AccNum],[Value_transaction],[SCA_id]) VALUES('2019-07-25 00:00:00','2019-07-25 00:00:00','FOLGELASTSCHRIFT','D01-8332720-6610260 Prime Video 1PJUZ9P3PYJZEDKI ','AMAZON INSTANT VIDEO GERMANY GMBH','DE07300308800013011001','-4.0','None')


INSERT INTO STR([Date_booked],[Date_ordered],[Text_transaction],[Use],[Contact],[AccNum],[Value_transaction],[SCA_id]) VALUES('2019-07-22 00:00:00','2019-07-22 00:00:00','EIGENE KREDITKARTENABRECHN.','VISA-ABR. 490762XXXXXX5575 ','KREDITKARTENABRECHNUNG','9902292060','-626.0','None')


INSERT INTO STR([Date_booked],[Date_ordered],[Text_transaction],[Use],[Contact],[AccNum],[Value_transaction],[SCA_id]) VALUES('2019-07-19 00:00:00','2019-07-19 00:00:00','BARGELDAUSZAHLUNG','2019-07-19T11:58 Debitk.5 2021-12 ','LUITPOLDST//SPARKASSE ERLANGEN/DE','DE68763500009000481431','-15.0','None')


INSERT INTO STR([Date_booked],[Date_ordered],[Text_transaction],[Use],[Contact],[AccNum]

...

INSERT INTO STR([Date_booked],[Date_ordered],[Text_transaction],[Use],[Contact],[AccNum],[Value_transaction],[SCA_id]) VALUES('2019-06-26 00:00:00','2019-06-26 00:00:00','FOLGELASTSCHRIFT','D01-3968535-6795828 Prime Video POO8MS7XFQNGQFII ','AMAZON INSTANT VIDEO GERMANY GMBH','DE07300308800013011001','-299.0','None')


INSERT INTO STR([Date_booked],[Date_ordered],[Text_transaction],[Use],[Contact],[AccNum],[Value_transaction],[SCA_id]) VALUES('2019-05-10 00:00:00','2019-05-10 00:00:00','KARTENZAHLUNG','2019-05-09T18:03 Debitk.5 2021-12 ','L Osteria Erlangen//Erlangen/DE','DE86300500000001052141','-26.4','None')


INSERT INTO STR([Date_booked],[Date_ordered],[Text_transaction],[Use],[Contact],[AccNum],[Value_transaction],[SCA_id]) VALUES('2019-06-25 00:00:00','2019-06-25 00:00:00','FOLGELASTSCHRIFT','D01-4898708-5723861 Prime Video 6L85GJFMCBLOPMNS ','AMAZON INSTANT VIDEO GERMANY GMBH','DE07300308800013011001','-299.0','None')


INSERT INTO STR([Date_booked],[Date_ordered],[Text_transac

INSERT INTO STR([Date_booked],[Date_ordered],[Text_transaction],[Use],[Contact],[AccNum],[Value_transaction],[SCA_id]) VALUES('2019-07-31 00:00:00','2019-08-01 00:00:00','ENTGELTABSCHLUSS','Entgeltabrechnung siehe Anlage ','nan','0000000000','-4.5','None')


INSERT INTO STR([Date_booked],[Date_ordered],[Text_transaction],[Use],[Contact],[AccNum],[Value_transaction],[SCA_id]) VALUES('2019-09-17 00:00:00','2019-09-17 00:00:00','FOLGELASTSCHRIFT','PP.8381.PP . DANCARLIN, Ihr Einkauf bei DANCARLIN ','PayPal (Europe) S.a.r.l. et Cie., S.C.A.','DE88500700100175526303','-561.0','None')


INSERT INTO STR([Date_booked],[Date_ordered],[Text_transaction],[Use],[Contact],[AccNum],[Value_transaction],[SCA_id]) VALUES('2019-07-31 00:00:00','2019-07-31 00:00:00','FOLGELASTSCHRIFT','D01-4144659-3352624 Prime Video 7JA5D1DOZ9LWKC6A ','AMAZON INSTANT VIDEO GERMANY GMBH','DE07300308800013011001','-3.98','None')


INSERT INTO STR([Date_booked],[Date_ordered],[Text_transaction],[Use],[Contact],[AccNum],[Va

INSERT INTO STR([Date_booked],[Date_ordered],[Text_transaction],[Use],[Contact],[AccNum],[Value_transaction],[SCA_id]) VALUES('2019-05-31 00:00:00','2019-06-01 00:00:00','ENTGELTABSCHLUSS','Entgeltabrechnung siehe Anlage ','nan','0000000000','-4.5','None')


INSERT INTO STR([Date_booked],[Date_ordered],[Text_transaction],[Use],[Contact],[AccNum],[Value_transaction],[SCA_id]) VALUES('2019-07-09 00:00:00','2019-07-09 00:00:00','FOLGELASTSCHRIFT','. SPOTIFY, Ihr Einkauf bei SPOTIFY ','PayPal (Europe) S.a.r.l. et Cie., S.C.A.','DE88500700100175526303','-999.0','None')


INSERT INTO STR([Date_booked],[Date_ordered],[Text_transaction],[Use],[Contact],[AccNum],[Value_transaction],[SCA_id]) VALUES('2019-05-31 00:00:00','2019-05-31 00:00:00','FOLGELASTSCHRIFT','D01-5935447-4453454 Prime Video 2TUXSL9CDFK0YASO ','AMAZON INSTANT VIDEO GERMANY GMBH','DE07300308800013011001','-2.99','None')


INSERT INTO STR([Date_booked],[Date_ordered],[Text_transaction],[Use],[Contact],[AccNum],[Value_transaction

INSERT INTO STR([Date_booked],[Date_ordered],[Text_transaction],[Use],[Contact],[AccNum],[Value_transaction],[SCA_id]) VALUES('2019-04-08 00:00:00','2019-04-08 00:00:00','KARTENZAHLUNG','2019-04-05T18:09 Debitk.5 2021-12 ','REWE SAGT DANKE. 46659170//Erlangen/DE','DE41300500000001484310','-847.0','None')


INSERT INTO STR([Date_booked],[Date_ordered],[Text_transaction],[Use],[Contact],[AccNum],[Value_transaction],[SCA_id]) VALUES('2019-04-08 00:00:00','2019-04-08 00:00:00','KARTENZAHLUNG','2019-04-06T20:15 Debitk.5 2021-12 ','IRISH CASTLE PUB GIR 79973149//NUERNBERG/DE','DE19300500000001458710','-2500.0','None')


INSERT INTO STR([Date_booked],[Date_ordered],[Text_transaction],[Use],[Contact],[AccNum],[Value_transaction],[SCA_id]) VALUES('2019-04-04 00:00:00','2019-04-04 00:00:00','ONLINE-UEBERWEISUNG','Mit Danke zurueck DATUM 04.04.2019, 09.32 UHR1.TAN 556059 ','Angelika Deinhard','DE60773501100000932426','-21000.0','None')


INSERT INTO STR([Date_booked],[Date_ordered],[Text_transact

INSERT INTO STR([Date_booked],[Date_ordered],[Text_transaction],[Use],[Contact],[AccNum],[Value_transaction],[SCA_id]) VALUES('2018-11-29 00:00:00','2018-11-29 00:00:00','DAUERAUFTRAG','Miete fuer Haesschen, du bist toll :) ','Elaine Fernandez','DE46763500000044116613','-23500.0','None')


INSERT INTO STR([Date_booked],[Date_ordered],[Text_transaction],[Use],[Contact],[AccNum],[Value_transaction],[SCA_id]) VALUES('2018-11-28 00:00:00','2018-11-28 00:00:00','KARTENZAHLUNG','2018-11-27T17:07 Debitk.5 2021-12 ','1283 Klier//Erlangen/DE','DE13269410530682240708','-2600.0','None')


INSERT INTO STR([Date_booked],[Date_ordered],[Text_transaction],[Use],[Contact],[AccNum],[Value_transaction],[SCA_id]) VALUES('2018-11-28 00:00:00','2018-11-28 00:00:00','FOLGELASTSCHRIFT','BEITRAG BIS 11/18 MANDATSREF 4005172401 GLAEUBIGER-ID DE71ZZZ00000053593 NAECHSTER BEITRAGSEINZUG IN BEKANNTER HOEHE UND VEREINBARTEM TURNUS ','IGM Erlangen','DE28500500000083044008','-205.0','None')


INSERT INTO STR([Date_b